In [ ]:
!pip install -q ipython-sql
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');


CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');


CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

 * sqlite://
(sqlite3.OperationalError) table sales already exists
[SQL: CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


**Case Study Questions**

* 1. What is the total amount each customer spent at the restaurant?
* 2. How many days has each customer visited the restaurant?
* 3. What was the first item from the menu purchased by each customer?
* 4. What is the most purchased item on the menu and how many times was it purchased by all customers?
* 5. Which item was the most popular for each customer?
* 6. Which item was purchased first by the customer after they became a member?
* 7. Which item was purchased just before the customer became a member?
* 8. What is the total items and amount spent for each member before they became a member?
* 9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
* 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?


In [ ]:
%%sql
SELECT * FROM sales

 * sqlite://
Done.


customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


In [ ]:
%%sql
SELECT * FROM menu

 * sqlite://
Done.


product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


In [ ]:
%%sql
SELECT * FROM members

 * sqlite://
Done.


customer_id,join_date
A,2021-01-07
B,2021-01-09


## **Total Amount Spent by Each Customer:**

Calculated the total amount spent by each customer by joining the Sales table with the Menu table on the product_id column and grouping the results by customer ID while summing up the prices of purchased items.

In [ ]:
%%sql
SELECT s.customer_id, sum(price) AS total_amount_spent FROM menu m
JOIN sales s ON m.product_id = s.product_id
GROUP BY s.customer_id

 * sqlite://
Done.


customer_id,total_amount_spent
A,76
B,74
C,36


## **Number of Days Visited by Each Customer:**

Determined the number of distinct days each customer visited the restaurant by selecting the customer ID and counting the distinct order dates from the Sales table, grouping the results by customer ID.

In [ ]:
%%sql
SELECT customer_id, COUNT(DISTINCT order_date) AS days_visited FROM sales
GROUP BY customer_id

 * sqlite://
Done.


customer_id,days_visited
A,4
B,6
C,2


## **First Item Purchased by Each Customer:**

Identified the first item purchased by each customer by joining the Sales table with the Menu table on the product_id column, grouping the results by customer ID, and selecting the product name of the first item purchased.

In [ ]:
%%sql
SELECT s.customer_id, m.product_name FROM sales s
JOIN menu m ON m.product_id = s.product_id
GROUP BY customer_id;

 * sqlite://
Done.


customer_id,product_name
A,sushi
B,curry
C,ramen


## **Most Purchased Item on the Menu:**

Identified the most purchased item on the menu by joining the Sales table with the Menu table on the product_id column, grouping the results by product name, and ordering the results by purchase count in descending order, then limiting to the top item.

In [ ]:
%%sql
SELECT m.product_name, count(m.product_name) most_purch FROM sales s
JOIN menu m ON m.product_id = s.product_id
GROUP BY m.product_name
ORDER BY most_purch DESC
LIMIT 1

 * sqlite://
Done.


product_name,most_purch
ramen,8


## **Most Popular Item for Each Customer:**

Determined the most popular item for each customer by joining the Sales table with the Menu table on the product_id column, grouping the results by customer ID and product name, and ordering the results by purchase count in descending order.

In [ ]:
%%sql
SELECT s.customer_id,m.product_name, COUNT(m.product_name) AS p_count FROM sales s
JOIN menu m ON m.product_id = s.product_id
GROUP BY s.customer_id, m.product_name
ORDER BY s.customer_id, p_count
LIMIT 2,5

 * sqlite://
Done.


customer_id,product_name,p_count
A,ramen,3
B,curry,2
B,ramen,2
B,sushi,2
C,ramen,3


## **First Item Purchased After Becoming a Member:**

Identified the first item purchased by each customer after they became a member by joining the Sales table with the Menu table on the product_id column, filtering for purchases after the join date of the member, and grouping the results by customer ID.

In [ ]:
%%sql
SELECT DISTINCT s.customer_id, m.product_name,s.order_date, m1.join_date FROM sales s
JOIN menu m ON s.product_id = m.product_id
JOIN members m1 ON s.customer_id = m1.customer_id
WHERE s.order_date >= m1.join_date
GROUP BY m1.customer_id
ORDER BY s.order_date

 * sqlite://
Done.


customer_id,product_name,order_date,join_date
A,curry,2021-01-07,2021-01-07
B,sushi,2021-01-11,2021-01-09


In [ ]:
%%sql
SELECT s.customer_id, m.product_name,s.order_date, m1.join_date FROM sales s
JOIN menu m ON s.product_id = m.product_id
JOIN members m1 ON s.customer_id = m1.customer_id
WHERE (s.customer_id == m1.customer_id AND s.order_date > m1.join_date)
GROUP BY m1.customer_id
ORDER BY s.order_date

 * sqlite://
Done.


customer_id,product_name,order_date,join_date
A,ramen,2021-01-10,2021-01-07
B,sushi,2021-01-11,2021-01-09


## **Item Purchased Just Before Becoming a Member:**

Identified the item purchased just before each customer became a member by joining the Sales table with the Menu table on the product_id column, filtering for purchases before the join date of the member, and grouping the results by customer ID.

In [ ]:
%%sql
SELECT s.customer_id, m.product_name,s.order_date, m1.join_date FROM sales s
JOIN menu m ON s.product_id = m.product_id
JOIN members m1 ON s.customer_id = m1.customer_id
WHERE (s.order_date < m1.join_date)
GROUP BY m1.customer_id
ORDER BY s.order_date

 * sqlite://
Done.


customer_id,product_name,order_date,join_date
A,sushi,2021-01-01,2021-01-07
B,curry,2021-01-01,2021-01-09


## **Total Items and Amount Spent Before Becoming a Member:**

Calculated the total items and amount spent for each customer before they became a member by joining the Sales table with the Menu table on the product_id column, filtering for purchases before the join date of the member, and grouping the results by customer ID while summing up the prices of purchased items.

In [ ]:
%%sql
SELECT s.customer_id, m.product_name,s.order_date, m1.join_date, sum(price) as total_spent FROM sales s
JOIN menu m ON s.product_id = m.product_id
JOIN members m1 ON s.customer_id = m1.customer_id
WHERE (s.order_date < m1.join_date)
GROUP BY m1.customer_id
ORDER BY s.order_date

 * sqlite://
Done.


customer_id,product_name,order_date,join_date,total_spent
A,sushi,2021-01-01,2021-01-07,25
B,curry,2021-01-01,2021-01-09,40


## **Points Earned by Each Customer:**

Calculated the points earned by each customer based on their purchases, where each $1 spent equates to 10 points and sushi has a 2x points multiplier. Joined the Sales table with the Menu table on the product_id column and applied the points multiplier for sushi purchases.

In [ ]:
%%sql
select customer_id,
sum(case
    when product_name = 'sushi' then 20 * price
    else 10 * price
end) total_points
from sales s
join menu m
  on s.product_id = m.product_id
group by customer_id;

 * sqlite://
Done.


customer_id,total_points
A,860
B,940
C,360


## **Points Earned by Customers After Joining the Program:**

Calculated the points earned by customers A and B in the first week after joining the program, where they earn double points on all items. Joined the Sales table with the Menu table on the product_id column and applied the points multiplier for purchases made within the first week after joining the program.

In [ ]:
%%sql
SELECT customer_id,
 SUM(CASE
  WHEN order_date >= join_date AND order_date < DATE(join_date, '+7 days') THEN 20 * price ELSE 10 * price
  END) AS total_points
FROM ( SELECT s.customer_id,s.order_date,m.price,mem.join_date FROM sales s
  JOIN menu m ON s.product_id = m.product_id
  LEFT JOIN members mem ON s.customer_id = mem.customer_id
  WHERE s.order_date >= '2021-01-01' AND s.order_date < '2021-01-31' AND (s.customer_id = 'A' OR s.customer_id = 'B'))
GROUP BY customer_id;

 * sqlite://
Done.


customer_id,total_points
A,1270
B,720
